In [1]:
import huggingface_hub
import os

from bitmind.synthetic_image_generation.synthetic_image_generator import SyntheticImageGenerator
from bitmind.image_dataset import ImageDataset
from bitmind.utils.data import sample_dataset_index_name
from bitmind.constants import DATASET_META, HUGGINGFACE_CACHE_DIR

2024-09-18 20:07:47.952217: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-18 20:07:47.976914: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-18 20:07:48.001908: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-18 20:07:48.009464: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-18 20:07:48.031086: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print("Loading real datasets")
real_image_datasets = [
    ImageDataset(ds['path'], 'train', ds.get('name', None), ds['create_splits'], 'REUSE_CACHE_IF_EXISTS')
    for ds in DATASET_META['real'][:1]
]


Loading real datasets
Downloaded bitmind/open-images-v7 dataset to /home/user/.cache/huggingface


In [3]:
synthetic_image_generator = SyntheticImageGenerator(
    prompt_type='annotation', 
    use_random_diffuser=False, 
    diffuser_name='black-forest-labs/FLUX.1-dev')

In [ ]:
import matplotlib.pyplot as plt

times = []

for _ in range(5):
    real_dataset_index, source_dataset = sample_dataset_index_name(real_image_datasets)
    real_dataset = real_image_datasets[real_dataset_index]
    images_to_caption, image_indexes = real_dataset.sample(k=1)  # [{'image': PIL Image ,'id': int}, ...]
    
    # generate captions for the real images, then synthetic images from these captions
    sample = synthetic_image_generator.generate(
        k=1, real_images=images_to_caption)[0]  # {'prompt': str, 'image': PIL Image ,'id': int}

    print(sample['prompt'])
    plt.imshow(sample['image'])
    plt.show()

    times.append(sample['gen_time'])

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Annotation loading: 59.19315552711487
Annotation gen: 4.476066589355469


Keyword arguments {'add_watermarker': False} are not expected by FluxPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Diffuser loading: 67.2234160900116
in generate_args
{'guidance_scale': 2, 'num_inference_steps': 100, 'generator': <torch._C.Generator object at 0x7f9e32b02370>, 'height': 512, 'width': 512}


In [ ]:
import numpy as np
np.mean(times), np.std(times)

In [ ]:
print('done')


In [ ]:
sample

In [ ]:
sample